In [24]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [25]:
#input_file = '/content/drive/My Drive/Study/T2/Bio/Project/Deep Learning with Python/maize_upreg_1000.fasta'
input_file = '/content/drive/My Drive/Deep Learning with Python/maize_upreg_1000.fasta'
handler = open(input_file,'r')
list_string=[]
y_train1 = []
sequence=""
count=0
start_flag=False;
for string in handler:
    if(string[0]=='>'):
        start_flag=True;
        length=len(sequence)
        if(sequence!="" and length>100):
            temp=0
            for char in sequence:
                if((char=='A' or char=='T' or char=='C' or char=='G')==False):
                    temp=1;
                    break
            if(temp==0):
                list_string.append(sequence)
                y_train1.append(0)
        sequence=""
        count+=1
        continue
    if(start_flag==True):
        sequence+=string[0:len(string)-1]
    
        
#input_file = '/content/drive/My Drive/Study/T2/Bio/Project/Deep Learning with Python/maize_downreg_1000.fasta'
input_file = '/content/drive/My Drive/Deep Learning with Python/maize_downreg_1000.fasta'
handler = open(input_file,'r')
line=0;
sequence=""
count=0
start_flag=False;
for string in handler:
    if(string[0]=='>'):
        start_flag=True;
        length=len(sequence)
        if(sequence!="" and length>100):
            #print(length)
            temp=0
            for char in sequence:
                if((char=='A' or char=='T' or char=='C' or char=='G')==False):
                    temp=1;
                    break
            if(temp==0):
                list_string.append(sequence)
                y_train1.append(0)
        sequence=""
        count+=1
        continue
    if(start_flag==True):
        sequence+=string[0:len(string)-1]

    
        
stresslen= len(list_string)
#input_file = '/content/drive/My Drive/Study/T2/Bio/Project/Deep Learning with Python/maize_true_const_1000.fasta'
input_file = '/content/drive/My Drive/Deep Learning with Python/maize_true_const_1000.fasta'
handler = open(input_file,'r')

line=0;
sequence=""
count=0
start_flag=False;
for string in handler:
    if(string[0]=='>'):
        start_flag=True;
        length=len(sequence)
        if(sequence!="" and length>100):
            #print(length)
            temp=0
            for char in sequence:
                if((char=='A' or char=='T' or char=='C' or char=='G')==False):
                    temp=1;
                    break
            if(temp==0):
                list_string.append(sequence)
                y_train1.append(1)
        sequence=""
        count+=1
        continue
    if(start_flag==True):
        sequence+=string[0:len(string)-1]
#     if(len(list_string)==2*stresslen):
#         break;
constlen = len(list_string)        
print(constlen-stresslen)
print(stresslen)

31532
9481


In [26]:
maxlen = 1000
import numpy as np
x_train1 = np.zeros((len(list_string),maxlen))
i=0
for string in list_string:
  j=0
  for char in string:
    if(char=='A'):
      x_train1[i][j]=1
    if(char=='T'):
      x_train1[i][j]=2
    if(char=='C'):
      x_train1[i][j]=3
    if(char=='G'):
      x_train1[i][j]=4
    j=j+1
  i=i+1
print(len(list_string[0]))
print(x_train1.shape)


1000
(41013, 1000)


In [27]:
y_train1 = np.array(y_train1)

indices = np.arange(x_train1.shape[0])
np.random.shuffle(indices)
x_train1 = x_train1[indices]
y_train1 = y_train1[indices]
print(x_train1.shape)
print(y_train1.shape)

(41013, 1000)
(41013,)


In [28]:
val_size = 2000
x_train = np.zeros((len(list_string)-2*val_size,maxlen))
y_train = np.random.rand(len(list_string)-2*val_size)
x_val = np.zeros((2*val_size,maxlen))
y_val = np.random.rand(2*val_size)

j=0
k=0
l=0
m=0
for i in range(0,len(list_string)):
  if(y_train1[i]==0 and j<val_size):
    x_val[l]= x_train1[i]
    y_val[l]= 0
    j+=1
    l+=1
  elif(y_train1[i]==1 and k<val_size):
    x_val[l]= x_train1[i]
    y_val[l]= 1
    k+=1
    l+=1
  else:
    x_train[m]=x_train1[i]
    y_train[m]=y_train1[i]
    m+=1

print(i,j,k,l,m)
    
    

41012 2000 2000 4000 37013


In [29]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(random_state=42)
x_train, y_train = sm.fit_resample(x_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train))

Original dataset shape Counter({1.0: 29532, 0.0: 7481})
Resampled dataset shape Counter({1.0: 29532, 0.0: 29532})


In [31]:
import numpy as np
x_train = np.round(x_train)

y_train = np.round(y_train)


array([1., 1., 1., ..., 0., 0., 0.])

In [32]:
import numpy as np
from keras import Input
from keras.models import Sequential,Model
from keras.layers import Embedding, Flatten, Dense,LSTM,MaxPooling1D,Conv1D,Dropout,merge,Permute,Lambda,Reshape,RepeatVector,Multiply,Activation
from keras import layers
from keras import backend as K
max_words = 5
embedding_dim = 4

units = 64

#functional model
embedding_matrix=np.array([[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0]])
_input = Input(shape=[maxlen], dtype='int32')
embedded = Embedding(max_words,embedding_dim,input_length=maxlen,weights=[embedding_matrix],trainable=False)(_input)
#embedded = Reshape((250,units))(embedded)
conv1 = layers.Conv1D(16, 7, activation='relu')(embedded)
maxpool1 = layers.MaxPooling1D(2)(conv1)
conv2 = layers.Conv1D(32, 7, activation='relu')(maxpool1)
maxpool2 = layers.MaxPooling1D(2)(conv2)
conv3 = layers.Conv1D(64, 7, activation='relu')(maxpool2)
maxpool3 = layers.MaxPooling1D(2)(conv3)
lstm = LSTM(units,return_sequences=True)(maxpool3)

attention = Dense(1, activation='tanh')(lstm)
attention = Flatten()(attention)
attention = Activation('softmax')(attention)
attention = RepeatVector(units)(attention)
attention = Permute([2, 1])(attention)

sent_representation = Multiply()([lstm, attention])
sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(units,))(sent_representation)

# flat = Flatten()(embedded)
# dense0 = Dense(256, activation='relu')(flat)
# drop0 = Dropout(0.3)(dense0)
# dense = Dense(128, activation='relu')(drop0)
# drop = Dropout(0.3)(dense)
# dense1 = Dense(64, activation='relu')(drop)
# drop1 = Dropout(0.3)(dense1)
# dense2 = Dense(32, activation='relu')(drop1)
# drop2 = Dropout(0.3)(dense2)
# output1 = Dense(1, activation='sigmoid')(drop2)
output1 = Dense(1, activation='sigmoid')(sent_representation)
model = Model(input=_input, output=output1)

#sequential model
# embedding_matrix=np.array([[0, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0]])

# model = Sequential()
# model.add(Embedding(max_words, embedding_dim,input_length=maxlen))
# model.add(layers.Conv1D(16, 7, activation='relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(32, 7, activation='relu'))
# model.add(layers.MaxPooling1D(2))
# model.add(layers.Conv1D(64, 7, activation='relu'))
# model.add(LSTM(64))
# # model.add(LSTM(64,return_sequences=True))
# # model.add(layers.MaxPooling1D(3))
# # model.add(Flatten())
# # model.add(Dense(64, activation='relu'))
# # model.add(Dropout(0.3))
# model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(1, activation='sigmoid'))

# model.layers[0].set_weights([embedding_matrix])
# model.layers[0].trainable = False


model.summary()







Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 4)      20          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 994, 16)      464         embedding_1[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 497, 16)      0           conv1d_1[0][0]                   
______________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [33]:
model.compile(optimizer='adam',
loss='binary_crossentropy',
metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=200,
#                     batch_size=256,validation_split =.1)
                      batch_size=256,validation_data =(x_val,y_val))
#model.save('/content/drive/My Drive/Study/T2/Bio/Project/Deep Learning with Python/my_model.h5')
model.save('/content/drive/My Drive/Deep Learning with Python/my_model_19_8.h5')


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 59064 samples, validate on 4000 samples
Epoch 1/200
59064/59064 [==============================] - 61s 1ms/step - loss: 0.6003 - acc: 0.6346 - val_loss: 0.7195 - val_acc: 0.5030
Epoch 2/200
59064/59064 [==============================] - 57s 960us/step - loss: 0.5845 - acc: 0.6493 - val_loss: 0.7173 - val_acc: 0.5005
Epoch 3/200
59064/59064 [==============================] - 57s 966us/step - loss: 0.5822 - acc: 0.6513 - val_loss: 0.7173 - val_acc: 0.5048
Epoch 4/200
59064/59064 [==============================] - 57s 965us/step - loss: 0.5798 - acc: 0.6547 - val_loss: 0.7141 - val_acc: 0.5048
Epoch 5/200
59064/59064 [==============================] - 57s 961us/step - loss: 0.5762 - acc: 0.6573 - val_loss: 0.6977 - val_acc: 0.5150
Epoch 6/200
59064/59064 [==============================] - 57s 958us/step - loss: 0.5713 - acc: 0.6635 - val_loss: 0.7175 - val_acc: 0.5198
Epoch 7/200
59064/